In [168]:
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from skimage import io
import glob
import numpy as np
import tqdm
import matplotlib.pyplot as plt

## Building a simple Convolutional Neural Net to classify webcam images

In [169]:
path_to_data = "./"

In [213]:
def create_dataset(path_to_data = "./data"):
    """
    Load all training data in the path and build the featureset and target column.
    """
    images = glob.glob(f"{path_to_data}/**/*.jpg")
    X = []
    y = []
    for image in tqdm.tqdm(images):
        im = io.imread(image)
        label = image.split("/")[-2]
        X.append(im)
        y.append(label)
        
    labels = y
    le = LabelEncoder()
    y = keras.utils.to_categorical(le.fit_transform(labels))
    X, y = shuffle(X,y)
    X = np.array(X).reshape(len(X), 60, 80, 1)
    X_train, X_test, y_train, y_test = train_test_split(X, y)
    return X_train, X_test, y_train, y_test, le
        
    

In [214]:
X_train, X_test, y_train, y_test, le = create_dataset()

100%|██████████| 1200/1200 [00:00<00:00, 2674.14it/s]


In [215]:
def create_model(output_size):
    """
    Create a simple convnet with keras
    """
    model = keras.models.Sequential()
    model.add(keras.layers.Conv2D(64, kernel_size = 3, activation = "relu", input_shape = (60,80,1)))
    model.add(keras.layers.Dropout(.2))
    model.add(keras.layers.Conv2D(32, kernel_size = 3, activation = "relu"))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dropout(.2))
    model.add(keras.layers.Dense(output_size, activation = "softmax"))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [216]:
#create and train the model
model = create_model(len(le.classes_))
model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = 5)

In [194]:
#save the model and classes
model.save("model_home")
classes = le.classes_
np.save("classes.npy", classes)

INFO:tensorflow:Assets written to: model_home/assets
